In [1]:
#Import libraries
import pandas as pd
import pymongo
import csv
import json
import numpy as np


In [2]:
#define de variable for each csv 
expo="resources/Exportacion.csv"
hibridos="resources/Vehi_hibrido_electricos.csv"
ventas="resources/Ventas.csv"
ventas2="resources/Ventas2R.csv"


In [3]:
#create the DF for each CSV
salesDF=pd.read_csv(ventas)
#salesDF.head()

In [4]:
salesDF.rename(columns={"Tema":"TypeOfSale","Año":"Year","Marca":"Brand","Mes":"Month","Modelo":"Model","Segmento":"Segment","Origen":"Origin","Tipo":"CarType","País origen":"OriginCountry"},inplace=True)
#salesDF.head()

In [5]:
#create the DF for each CSV
########## Second graphic#############
salesDF2=pd.read_csv(ventas2,encoding='latin-1')
#salesDF2.head()

In [6]:
ExpoDF=pd.read_csv(expo)
#ExpoDF.head()

In [7]:
ExpoDF.rename(columns={"Tema":"TypeOfSale","Año":"Year","Marca":"Brand","Mes":"Month","Modelo":"Model","Segmento":"Segment","Tipo":"CarType","PaisDestino":"DestinationCountry"},inplace=True)
#ExpoDF.head()

In [8]:
#DF for Hybrid and electric cars in Mexico (INEGI information)
HybrDF=pd.read_csv(hibridos,encoding='latin-1')
#HybrDF.head()

In [9]:
HybrDF.rename(columns={"Tema":"TypeOfSale","Año":"Year","Mes":"Month","EntidadFederativa":"State","UnidadesVehicularesElectricas":"ElectricCars","UnidadesVehicularesHibridasPlugin1":"HybridPlugin1","UnidadesVehicularesHibridas2":"HybridPlugin2"},inplace=True)
#HybrDF.head()

In [10]:
######## HYBRID'S SALES DATAFRAME BY YEAR #####################

HybByYear=HybrDF[["Year","TypeOfSale","State","ElectricCars","HybridPlugin1","HybridPlugin2","Total"]]

SaleByType=HybByYear.groupby(["Year","State"])
HybTotal=SaleByType["Total"].sum()
HybridByStateDF=pd.DataFrame({
    
    "Total_Sales":HybTotal
    
})
HybridByStateDF.reset_index(inplace=True)
#HybridByStateDF.head(15)


In [13]:
######## SEGMENT DATAFRAME BY BRAND AND YEAR FOR SECOND GRAPHIC#####################
TotalbyYear2=salesDF2[["Segmento","Date","Total"]]

SaleByBrand2=TotalbyYear2.groupby(["Segmento","Date"])
BrandTotal2=SaleByBrand2["Total"].sum()
ByBrandDF2=pd.DataFrame({
    
    "Total_Sales":BrandTotal2
})
ByBrandDF2.reset_index(inplace=True)

#ByBrandDF2.tail(15)


In [14]:
######## SALES DATAFRAME BY BRAND AND YEAR #####################
TotalbyYear=salesDF[["Year","Brand","Total"]]

SaleByBrand=TotalbyYear.groupby(["Year","Brand"])
BrandTotal=SaleByBrand["Total"].sum()
ByBrandDF=pd.DataFrame({
    
    "Total_Sales":BrandTotal
})
ByBrandDF.reset_index(inplace=True)

#ByBrandDF.head(15)


In [15]:
conn="mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [16]:
#create the database in Mongo
db = client['FinalProject']


In [17]:
for i in range(HybrDF.shape[0]):
    document = HybrDF.iloc[i].to_dict()
    document = {x:str(y) for x,y in document.items()}
    db.Hybrid.insert_one(document)

In [18]:
for i in range(ExpoDF.shape[0]):
    document2 = ExpoDF.iloc[i].to_dict()
    document2 = {x:str(y) for x,y in document2.items()}
    db.expo.insert_one(document2)

In [19]:
for i in range(salesDF.shape[0]):
    document3 = salesDF.iloc[i].to_dict()
    document3 = {x:str(y) for x,y in document3.items()}
    db.sales.insert_one(document3)

In [20]:
for i in range(HybridByStateDF.shape[0]):
    document4 = HybridByStateDF.iloc[i].to_dict()
    document4 = {x:str(y) for x,y in document4.items()}
    db.HybridFilter.insert_one(document4)

In [21]:
for i in range(ByBrandDF.shape[0]):
    document5 = ByBrandDF.iloc[i].to_dict()
    document5 = {x:str(y) for x,y in document5.items()}
    db.BrandFilter.insert_one(document5)


In [26]:
######################################
for i in range(ByBrandDF2.shape[0]):
    document7 = ByBrandDF2.iloc[i].to_dict()
    document7 = {x:str(y) for x,y in document7.items()}
    db.BrandFilter2.insert_one(document7)

In [23]:
from flask import jsonify

In [24]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["FinalProject"]
mycol = mydb["BrandFilter"]

resultado = mycol.find({"Year":"2006"},{"_id":False})
resultado= [x for i,x in enumerate(resultado)]
resultado

[{'Year': '2006', 'Brand': 'Acura', 'Total_Sales': '1631'},
 {'Year': '2006', 'Brand': 'Audi', 'Total_Sales': '4205'},
 {'Year': '2006', 'Brand': 'BMW', 'Total_Sales': '7087'},
 {'Year': '2006', 'Brand': 'Bentley', 'Total_Sales': '36'},
 {'Year': '2006', 'Brand': 'Chrysler', 'Total_Sales': '128446'},
 {'Year': '2006', 'Brand': 'Fiat', 'Total_Sales': '1040'},
 {'Year': '2006', 'Brand': 'Ford_Motor', 'Total_Sales': '177595'},
 {'Year': '2006', 'Brand': 'General_Motors', 'Total_Sales': '245090'},
 {'Year': '2006', 'Brand': 'Honda', 'Total_Sales': '47471'},
 {'Year': '2006', 'Brand': 'Jaguar', 'Total_Sales': '332'},
 {'Year': '2006', 'Brand': 'Land_Rover', 'Total_Sales': '774'},
 {'Year': '2006', 'Brand': 'Lincoln', 'Total_Sales': '5187'},
 {'Year': '2006', 'Brand': 'Mazda', 'Total_Sales': '7495'},
 {'Year': '2006', 'Brand': 'Mercedes_Benz', 'Total_Sales': '6452'},
 {'Year': '2006', 'Brand': 'Mini', 'Total_Sales': '1806'},
 {'Year': '2006', 'Brand': 'Mitsubishi', 'Total_Sales': '16751'},
 

In [25]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["FinalProject"]
mycol = mydb["sales"]

resultado = mycol.find()
resultado= [x for i,x in enumerate(resultado)]
resultado

[{'_id': ObjectId('5e603b12ce852a7217c08922'),
  'TypeOfSale': 'Venta_al_publico_de_vehiculos',
  'Year': '2005',
  'Month': 'Febrero',
  'Brand': 'General_Motors',
  'Model': '147',
  'CarType': 'Automoviles',
  'Segment': 'De_Lujo',
  'Origin': 'Importado',
  'OriginCountry': 'Italia',
  'Total': '22'},
 {'_id': ObjectId('5e603b12ce852a7217c08923'),
  'TypeOfSale': 'Venta_al_publico_de_vehiculos',
  'Year': '2005',
  'Month': 'Marzo',
  'Brand': 'General_Motors',
  'Model': '147',
  'CarType': 'Automoviles',
  'Segment': 'De_Lujo',
  'Origin': 'Importado',
  'OriginCountry': 'Italia',
  'Total': '12'},
 {'_id': ObjectId('5e603b12ce852a7217c08924'),
  'TypeOfSale': 'Venta_al_publico_de_vehiculos',
  'Year': '2005',
  'Month': 'Mayo',
  'Brand': 'General_Motors',
  'Model': '147',
  'CarType': 'Automoviles',
  'Segment': 'De_Lujo',
  'Origin': 'Importado',
  'OriginCountry': 'Italia',
  'Total': '12'},
 {'_id': ObjectId('5e603b12ce852a7217c08925'),
  'TypeOfSale': 'Venta_al_publico_de_